In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import json

### Pulizia Dataset
Questo Notebook contiene i codici estrapolati dai singoli notebook, commentati e ripuliti delle parti non necessarie al funzionamento.
Vi è inoltre una rappresentazione grafica dei dati.

### Recupero dataset aggiornati protezione civile

In [2]:
fetchDataset = json.load(open("Json/dataset.json", "r"))
#Recupera gli url dal JSON, scarica i file e li salva nelle cartelle specifiche.
#ATTENZIONE: potrebbe dare errori se l'url non è raggiungibile
for value in fetchDataset:
    pd.read_csv(value["url"], error_bad_lines=False).to_csv(value["path"]+value["name"], index=False)

### Dataset Francia

In [7]:
#Carico i csv
franciaDataset = pd.read_csv("Datasets/Francia/france.csv")

#Cancello le colonne che non ritengo utili, ovvero codice città/paese/regione e la provenienza del dato
franciaDataset.drop(columns=['maille_code','source_nom','source_url','source_type'],inplace=True)

#Raggruppo date-granularite(ovvero regione/stato/provincia/ecc.)-maille_nom per tenere in considerazione solo il massimo valore 
#(vi sono dati duplicati in quanto vi sono più fonti di provenienza dei dati) in questo modo si eliminano i duplicati
groupedFrancia = franciaDataset.groupby(['date','granularite','maille_nom'])[['cas_confirmes','deces','reanimation','hospitalises','gueris','depistes']].max()

#Riempio i NaN con 0, poichè mancano i dati relativi a contagiati,ecc.
groupedFrancia = groupedFrancia.fillna(0)

#Salvo il csv
groupedFrancia.to_csv('Datasets/Francia/france_clean.csv', index=True)

groupedFrancia.tail()

cas_confirmes  deces  \
date       granularite maille_nom                                         
2020-03-26 region      Nouvelle-Aquitaine                    0.0   34.0   
                       Occitanie                          1285.0   54.0   
                       Pays de la Loire                      0.0   24.0   
                       Provence-Alpes-Côte d'Azur            0.0   44.0   
                       Île-de-France                         0.0  460.0   

                                                   reanimation  hospitalises  \
date       granularite maille_nom                                              
2020-03-26 region      Nouvelle-Aquitaine                 96.0         399.0   
                       Occitanie                         177.0         551.0   
                       Pays de la Loire                   59.0         282.0   
                       Provence-Alpes-Côte d'Azur        164.0         695.0   
                       Île-de-France                    1269.0        5132.0   

                                                   gueris  depistes  
date       granularite maille_nom                                    
2020-03-26 region      Nouvelle-Aquitaine           163.0       0.0  
                       Occitanie                    233.0       0.0  
                       Pays de la Loire             127.0       0.0  
                       Provence-Alpes-Côte d'Azur   599.0       0.0  
                       Île-de-France               1310.0       0.0

### Dataset Kaggle COVID19

In [18]:
datasetCVD19_complete = pd.read_csv("Datasets/Kaggle/covid_19_clean_complete.csv")

#Verifico le colonne con campi nulli
null_counts = datasetCVD19_complete.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)
#print(null_counts)

#Elimino le colonne che non utilizzo
datasetCVD19_complete = datasetCVD19_complete.drop(columns="Lat")
datasetCVD19_complete = datasetCVD19_complete.drop(columns="Long")

#print(datasetCVD19_complete.dtypes)

#Modifico la colonna Date dandogli il formato m-d-y
datasetCVD19_complete["Date"] = pd.to_datetime(datasetCVD19_complete["Date"], format='%m/%d/%y')

#print(datasetCVD19_complete.dtypes)

#Popolo i campi della colonna provincia con i dati relativi allo stato (per quegli stati in cui l'analisi è a livello nazionale)
datasetCVD19_complete["Province/State"] = datasetCVD19_complete["Province/State"].fillna(datasetCVD19_complete["Country/Region"])

datasetCVD19_complete.to_csv("Datasets/Kaggle/covid_19_clean_complete_clean.csv",index=False)

datasetCVD19_complete.tail()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered
16050,Mali,Mali,2020-03-26,4,0,0
16051,Saint Kitts and Nevis,Saint Kitts and Nevis,2020-03-26,2,0,0
16052,Northwest Territories,Canada,2020-03-26,1,0,0
16053,Yukon,Canada,2020-03-26,3,0,0
16054,Kosovo,Kosovo,2020-03-26,71,1,0


In [19]:
open_line_list = pd.read_csv('Datasets/Kaggle/COVID19_open_line_list.csv')

#Rimuovo le colonne in più vuote, caricate da pandas
open_line_list = open_line_list.iloc[:,0:33]

#Recupero solamente le colonne di interesse
open_line_list = open_line_list[['age','sex','city','province','country','wuhan(0)_not_wuhan(1)','date_confirmation','symptoms','lives_in_Wuhan','travel_history_location','chronic_disease_binary','chronic_disease','outcome','date_death_or_discharge']]

#La colonna Chronic_disease_binary viene popolata con 0 dove i valori sono na, in quanto corrispondono ad una mancata presenza di malattie croniche
open_line_list['chronic_disease_binary']=open_line_list['chronic_disease_binary'].fillna(0)

#Creo una colonna binaria per individuare rapidamente se una persona è stata a Wuhan o no, in base ai dati di travel history location
open_line_list['travel_in_Wuhan']=open_line_list['travel_history_location'].str.contains("Wuhan",na=False, regex=True)

#Cambio la logica di wuhan(0)_not_wuhan(1), mettendo a 1 se è stato a wuhan, 0 altrimenti. Cambio il nome della colonna in "inWuhan"
open_line_list['wuhan(0)_not_wuhan(1)']=1-open_line_list['wuhan(0)_not_wuhan(1)']
open_line_list.rename(columns={"wuhan(0)_not_wuhan(1)":"inWuhan"},inplace=True)
open_line_list['inWuhan']=open_line_list['inWuhan'].fillna(0)

#Popolo i NaN con "no" nella colonna "lives_in_wuhan" e creo rispettiva colonna binaria
open_line_list['lives_in_Wuhan']=open_line_list['lives_in_Wuhan'].fillna('no')
open_line_list['lives_in_Wuhan_binary']=open_line_list['lives_in_Wuhan']!='no'

#Elimino le righe in cui "country" è nullo
indexOLL = open_line_list[open_line_list['country'].isnull()].index
open_line_list.drop(indexOLL,inplace=True)

#Elimino le colonne che non sono più utili, in quanto o troppo poco presenti o sostituite da colonne binarie
open_line_list.drop(columns=["lives_in_Wuhan","travel_history_location","chronic_disease","outcome","date_death_or_discharge"],inplace=True)

#Rendo vuoti i campi "age" e "date_confirmation" nei casi in cui sono presenti dei range e non valori precisi
indexesOLL = open_line_list[open_line_list['age'].str.contains("-",na=True, regex=None)==True].index
open_line_list.loc[indexesOLL.tolist(),'age'] = ''
indexesOLL = open_line_list[open_line_list['date_confirmation'].str.contains("-",na=True, regex=None)==True].index
open_line_list.loc[indexesOLL.tolist(),'date_confirmation'] = ''

open_line_list.to_csv('Datasets/Kaggle/COVID19_open_line_list_clean.csv',index=False)

open_line_list.tail()

,age,sex,city,province,country,inWuhan,date_confirmation,symptoms,chronic_disease_binary,travel_in_Wuhan,lives_in_Wuhan_binary
13169,,male,Nagoya City,Aichi Prefecture,Japan,0.0,28.02.2020,fever (38 ° C),0.0,False,False
13170,,female,NaN,Osaka,Japan,0.0,28.02.2020,fever (38 ° C),0.0,False,False
13171,,female,NaN,Osaka,Japan,0.0,28.02.2020,NaN,0.0,False,False
13172,,female,Amsterdam,NaN,Netherlands,0.0,28.02.2020,mild,0.0,False,False
13173,,female,"Jose Joaquin de Olmedo Airport, Guayaquil",Guayas,Ecuador,0.0,29.02.2020,fever,0.0,False,False


In [20]:
time_series_confirmed = pd.read_csv("Datasets/Kaggle/time_series_covid_19_confirmed.csv")
time_series_death = pd.read_csv("Datasets/Kaggle/time_series_covid_19_deaths.csv")
time_series_recovered = pd.read_csv("Datasets/Kaggle/time_series_covid_19_recovered.csv")

#Elimino le colonne Lat e Long dai dataframe
time_series_confirmed = time_series_confirmed.drop(columns="Lat")
time_series_confirmed = time_series_confirmed.drop(columns="Long")
time_series_death = time_series_death.drop(columns="Lat")
time_series_death = time_series_death.drop(columns="Long")
time_series_recovered = time_series_recovered.drop(columns="Lat")
time_series_recovered = time_series_recovered.drop(columns="Long")

#Popolo i campi della colonna provincia con i dati relativi allo stato (per quegli stati in cui l'analisi è a livello nazionale)
time_series_confirmed["Province/State"] = time_series_confirmed["Province/State"].fillna(time_series_confirmed["Country/Region"])
time_series_death["Province/State"] = time_series_death["Province/State"].fillna(time_series_death["Country/Region"])
time_series_recovered["Province/State"] = time_series_recovered["Province/State"].fillna(time_series_recovered["Country/Region"])

time_series_confirmed.to_csv("Datasets/Kaggle/time_series_covid_19_confirmed_clean.csv",index=False)
time_series_death.to_csv("Datasets/Kaggle/time_series_covid_19_deaths_clean.csv",index=False)
time_series_recovered.to_csv("Datasets/Kaggle/time_series_covid_19_recovered_clean.csv",index=False)

time_series_confirmed.head()
time_series_death.head()
time_series_recovered.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20
0,Afghanistan,Afghanistan,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,2
1,Albania,Albania,0,0,0,0,0,0,0,0,...,0,0,0,0,2,2,2,10,17,17
2,Algeria,Algeria,0,0,0,0,0,0,0,0,...,12,12,32,32,32,65,65,24,65,29
3,Andorra,Andorra,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,Angola,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Dataset GDP e Spesa Sanità

In [12]:
gdpDataset = pd.read_csv("Datasets/GDP/GDP_per_country.csv")

#Selezione solamente le colonne più significative
gdpDatasetClean = gdpDataset[['Country Name','Country Code', 'Indicator Name','Indicator Code','2016']].copy()

#Elimino le righe (quindi i paesi) per cui non ho dati nel 2016
gdpDatasetClean.dropna(subset=['2016'],inplace=True)
gdpDatasetClean.rename(columns={"2016":"GDP 2016"},inplace=True)

#Rinomino alcune nazioni in quanto scritte in modo diverso rispetto agli altri dataset elaborati
gdpDatasetClean['Country Name'].loc[(gdpDatasetClean['Country Name']=='United States')]='US'
gdpDatasetClean['Country Name'].loc[(gdpDatasetClean['Country Name']=='Korea, Rep.')]='South Korea'
gdpDatasetClean['Country Name'].loc[(gdpDatasetClean['Country Name']=='Iran, Islamic Rep.')]='Iran'

gdpDatasetClean.to_csv('Datasets/GDP/GDP_per_country_clean.csv', index=False)

healthDataset = pd.read_csv("Datasets/GDP/Spesa_sanita_perc_gdp.csv")

#Selezione solamente le colonne più significative
healthDatasetClean = healthDataset[['Country Name','Country Code','Indicator Name','Indicator Code','2016']].copy()

#Elimino le righe (quindi i paesi) per cui non ho dati nel 2016
healthDatasetClean.dropna(subset=['2016'],inplace=True)
healthDatasetClean.rename(columns={"2016":"HEALTH Perc 2016"},inplace=True)

#Rinomino alcune nazioni in quanto scritte in modo diverso rispetto agli altri dataset elaborati
healthDatasetClean['Country Name'].loc[(healthDatasetClean['Country Name']=='United States')]='US'
healthDatasetClean['Country Name'].loc[(healthDatasetClean['Country Name']=='Korea, Rep.')]='South Korea'
healthDatasetClean['Country Name'].loc[(healthDatasetClean['Country Name']=='Iran, Islamic Rep.')]='Iran'

healthDatasetClean.to_csv('Datasets/GDP/Spesa_sanita_perc_gdp_clean.csv', index=False)

gdp = pd.read_csv('Datasets/GDP/GDP_per_country_clean.csv')
health = pd.read_csv('Datasets/GDP/Spesa_sanita_perc_gdp_clean.csv')

#Modifico l'index dei df impostando il CountryCode (che è univoco ed è uno standard per l'identificazione delle nazioni)
gdp = gdp.set_index('Country Code')
health = health.set_index('Country Code')

#Unisco i due dataset
unified_gdp_health = pd.concat([gdp,health],axis=1)

#Rimuovo le colonne duplicate
unified_clean_gdp_health = unified_gdp_health.iloc[:,[0,3,7]].copy()

#Rimuovo eventuali nazioni che non hanno o il GDP o l'Health
unified_clean_gdp_health.dropna(subset=['GDP 2016','HEALTH Perc 2016'],how='any',inplace=True)

#Calcolo il costo in $ per la sanità
unified_clean_gdp_health['HEALTH 2016']=unified_clean_gdp_health['GDP 2016']*unified_clean_gdp_health['HEALTH Perc 2016']/100

#Ripristino gli indici
unified_clean_gdp_health.reset_index(inplace=True)
unified_clean_gdp_health.rename(columns={"index":"Country Code"},inplace=True)

unified_clean_gdp_health.to_csv('Datasets/GDP/GDP_HEALTH_per_country_clean.csv', index=False)

unified_clean_gdp_health.head()

,Country Code,Country Name,GDP 2016,HEALTH Perc 2016,HEALTH 2016
0,AFG,Afghanistan,1.936264e+10,10.198677,1.974733e+09
1,AGO,Angola,1.011239e+11,2.877825,2.910168e+09
2,ALB,Albania,1.186135e+10,6.697895,7.944610e+08
3,AND,Andorra,2.877312e+09,10.367728,2.983119e+08
4,ARB,Arab World,2.510521e+12,4.887915,1.227121e+11


### Dataset Istat

In [21]:
istatDataset = pd.read_csv("Datasets/Istat/DCIS_POPRES1_23032020145607847.csv")

#Elimino le colonne non utili
istatDataset = istatDataset.drop(["TIME", "Seleziona periodo", "Flag Codes", "Flags", "TIPO_DATO15", "Tipo di indicatore demografico"], axis=1)

istatDataset.to_csv('Datasets/Istat/istat_clean.csv', index=False)

istatDataset.head()

,ITTER107,Territorio,SEXISTAT1,Sesso,ETA1,Età,STATCIV2,Stato civile,Value
0,IT,Italia,1,maschi,Y0,0 anni,1,nubile/celibe,225428
1,IT,Italia,1,maschi,Y0,0 anni,99,totale,225428
2,IT,Italia,2,femmine,Y0,0 anni,1,nubile/celibe,212859
3,IT,Italia,2,femmine,Y0,0 anni,99,totale,212859
4,IT,Italia,9,totale,Y0,0 anni,1,nubile/celibe,438287


### Dataset Posti Nelle Terapie Intensive

In [22]:
#Specifichiamo il separatore del csv in quanto non è la più utilizzata ','
terapie_intensive_dataset = pd.read_csv("Datasets/TerapiaIntensiva/C_17_dataset_96_0_upFile.csv", encoding = "ISO-8859-1", sep=';')

#Estraggo solo le colonne di interesse
terapie_intensive = terapie_intensive_dataset[["Anno", "Descrizione Regione", "Descrizione disciplina", "Tipo di Disciplina", "Totale posti letto"]].copy()
#Rimuovo gli spazi presenti nelle tuple di alcune colonne
terapie_intensive = terapie_intensive.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#Estraggo le tuple con Descrizione disciplina uguale a
#PNEUMOLOGIA, MALATTIE INFETTIVE E TROPICALI, TERAPIA INTENSIVA,
#ovvero le terapie in cui potrebbero essere ricoverati i pazienti Cvid-19
terapie_intensive = terapie_intensive[terapie_intensive["Descrizione disciplina"].isin(["PNEUMOLOGIA", "MALATTIE INFETTIVE E TROPICALI", "TERAPIA INTENSIVA"])]

#Prendiamo l'anno 2018 in quanto è l'ultimo anno disponibile e
#quindi sono i dati più attendibili sull'effettivo numero di posti letto disponibili
terapie_intensive = terapie_intensive[terapie_intensive["Anno"] == 2018]

#Sommo i posti letto di una stessa regione raggruppando per Regione, descrizione e tipo disciplina
terapie_intensive = terapie_intensive.groupby(["Descrizione Regione", "Descrizione disciplina", "Tipo di Disciplina"], as_index=False).agg({"Totale posti letto": 'sum'})

terapie_intensive.to_csv("Datasets/TerapiaIntensiva/Totale_posti_terapie_intensive_per_regione.csv",index=False)

terapie_intensive.head()

,Descrizione Regione,Descrizione disciplina,Tipo di Disciplina,Totale posti letto
0,ABRUZZO,MALATTIE INFETTIVE E TROPICALI,ACUTI,87
1,ABRUZZO,PNEUMOLOGIA,ACUTI,72
2,ABRUZZO,TERAPIA INTENSIVA,ACUTI,109
3,BASILICATA,MALATTIE INFETTIVE E TROPICALI,ACUTI,46
4,BASILICATA,PNEUMOLOGIA,ACUTI,76


### Dataset Andamento Nazionale

In [3]:
pcNazionaleInputDataset = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-andamento-nazionale.csv")

# Rimozione delle colonne non necessarie
pcNazionaleOutputDataset = pcNazionaleInputDataset.drop(columns="stato")
pcNazionaleOutputDataset = pcNazionaleOutputDataset.drop(columns="note_it")
pcNazionaleOutputDataset = pcNazionaleOutputDataset.drop(columns="note_en")

# Riformattazione della data
pcNazionaleOutputDataset["data"] = pd.to_datetime(pcNazionaleOutputDataset["data"], format='%Y-%m-%d').dt.normalize()

pcNazionaleOutputDataset.to_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-andamento-nazionale_clean.csv",index=False)

pcNazionaleOutputDataset.head()

,data,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati
0,2020-02-24,101,26,127,94,221,0,221,1,7,229,4324,NaN
1,2020-02-25,114,35,150,162,311,90,93,1,10,322,8623,NaN
2,2020-02-26,128,36,164,221,385,74,78,3,12,400,9587,NaN
3,2020-02-27,248,56,304,284,588,203,250,45,17,650,12014,NaN
4,2020-02-28,345,64,409,412,821,233,238,46,21,888,15695,NaN


### Dataset Andamento Regionale

In [4]:
pcRegioniInputDataset = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-regioni.csv")

# Rimozione delle colonne non necessarie
pcRegioniOutputDataset = pcRegioniInputDataset.drop(columns="stato")
pcRegioniOutputDataset = pcRegioniOutputDataset.drop(columns="codice_regione")
pcRegioniOutputDataset = pcRegioniOutputDataset.drop(columns="lat")
pcRegioniOutputDataset = pcRegioniOutputDataset.drop(columns="long")
pcRegioniOutputDataset = pcRegioniOutputDataset.drop(columns="note_it")
pcRegioniOutputDataset = pcRegioniOutputDataset.drop(columns="note_en")

# Riformattazione della data
pcRegioniOutputDataset["data"] = pd.to_datetime(pcRegioniOutputDataset["data"], format='%Y-%m-%d').dt.normalize()

pcRegioniOutputDataset.to_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-regioni_clean.csv",index=False)

pcRegioniOutputDataset.head()

,data,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati
0,2020-02-24,Abruzzo,0,0,0,0,0,0,0,0,0,0,5,NaN
1,2020-02-24,Basilicata,0,0,0,0,0,0,0,0,0,0,0,NaN
2,2020-02-24,P.A. Bolzano,0,0,0,0,0,0,0,0,0,0,1,NaN
3,2020-02-24,Calabria,0,0,0,0,0,0,0,0,0,0,1,NaN
4,2020-02-24,Campania,0,0,0,0,0,0,0,0,0,0,10,NaN


### Dataset Andamento Provinciale

In [5]:
pcProvincieInputDataset = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-province.csv")

# Rimozione delle colonne non necessarie
pcProvincieOutputDataset = pcProvincieInputDataset.drop(columns="stato")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="codice_regione")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="codice_provincia")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="sigla_provincia")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="lat")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="long")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="note_it")
pcProvincieOutputDataset = pcProvincieOutputDataset.drop(columns="note_en")

# Rimozione delle righe con denominazione "In fase di definizione/aggiornamento"
indexNames = pcProvincieOutputDataset[pcProvincieOutputDataset['denominazione_provincia'] == "In fase di definizione/aggiornamento"].index

pcProvincieOutputDataset.drop(indexNames , inplace=True)

# Riformattazione della data
pcProvincieOutputDataset["data"] = pd.to_datetime(pcProvincieOutputDataset["data"], format='%Y-%m-%d').dt.normalize()

pcProvincieOutputDataset.to_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-province_clean.csv",index=False)

pcProvincieOutputDataset.head()

,data,denominazione_regione,denominazione_provincia,totale_casi
0,2020-02-24,Abruzzo,Chieti,0
1,2020-02-24,Abruzzo,L'Aquila,0
2,2020-02-24,Abruzzo,Pescara,0
3,2020-02-24,Abruzzo,Teramo,0
5,2020-02-24,Basilicata,Matera,0


### Dataset Temperature Per Nazione

In [6]:
temperatureInputDataset = pd.read_csv("Datasets/Temperature/GlobalLandTemperaturesByCountry.csv")

# Filtro il dataset di partenza prendendo solo le date a partire dall'anno 2000
filteredTemperatureDataset = temperatureInputDataset[temperatureInputDataset['dt'] > '2000-01-01']

# Applico un reset degli indici dopo il filtro sulla data e rimuovo la colonna riguardante il margine di errore
# sulla misurazione
filteredTemperatureDataset = filteredTemperatureDataset.reset_index()
filteredTemperatureDataset = filteredTemperatureDataset.drop(columns = 'index')
filteredTemperatureDataset = filteredTemperatureDataset.drop(columns = 'AverageTemperatureUncertainty')

# Liste di supporto per ciclare il dataset
countryList = filteredTemperatureDataset['Country'].unique()
monthList = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Creazione Dataframe di output
temperatureOutputDataset = pd.DataFrame({'Month': [], 'AverageTemperature': [], 'Country': []})

# Ciclo sul dataset per ogni paese estraendo la media di temperatura per ogni mese
# Codice commentato per l'elevato tempo di computazione, il valore di output viene popolato tramite l'import del 
# csv generato
#
# for country in countryList:
#     tmpDataset = filteredTemperatureDataset[filteredTemperatureDataset['Country'] == country]
#     for month in monthList:
#         # Utilizzo una regex per determinare il mese interessato
#         tmpMonthDf = temperatureInputDataset[temperatureInputDataset['dt'].str.match('\d\d\d\d-' + month + '-\d\d') == True]
#         temperatureMean = tmpMonthDf['AverageTemperature'].mean()
#         # Creazione row temporanea da appendere al Dataframe di output
#         tmpRow = pd.DataFrame({'Month': [month],'AverageTemperature': [temperatureMean], 'Country': [country]})
#         temperatureOutputDataset = temperatureOutputDataset.append(tmpRow, ignore_index = True)
#
# temperatureOutputDataset.to_csv("GlobalLandTemperatureByCountry(2000+).csv",index=False)

temperatureOutputDataset = pd.read_csv("Datasets/Temperature/GlobalLandTemperatureByCountry(2000+).csv")

temperatureOutputDataset.head()

,Month,AverageTemperature,Country
0,1,11.635348,Åland
1,2,12.577014,Åland
2,3,14.574245,Åland
3,4,16.969519,Åland
4,5,19.523826,Åland


### Trasformazione Dataset Regionale/Nazionale In Timeseries

In [7]:
datasetTerapiaTS = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-regioni_clean.csv")

#Recupero le colonne di interesse
TSterapia_regioni = datasetTerapiaTS[["data", "denominazione_regione", "terapia_intensiva"]].copy()

#Modifico l'indice, impostando la regione
TSterapia_regioni.set_index("denominazione_regione", inplace=True)

#Effettuo il pivot della tabella ottenendo la time series
TSterapia_regioni = TSterapia_regioni.pivot_table(index='denominazione_regione', columns='data', values='terapia_intensiva')

TSterapia_regioni.to_csv("Datasets/ProtezioneCivile/ts_regioni_terapia.csv")

TSterapia_regioni.head()

data,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,...,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17
denominazione_regione,,,,,,,,,,,,,,,,,,,,,
Abruzzo,0,0,0,0,0,0,0,0,0,0,...,10,10,10,9,8,7,6,6,7,6
Basilicata,0,0,0,0,0,0,0,0,0,0,...,2,1,1,1,1,1,1,1,1,1
Calabria,0,0,0,0,0,0,0,0,0,0,...,2,2,2,1,1,1,2,2,2,2
Campania,0,0,0,0,0,0,0,0,0,0,...,28,26,20,23,20,19,19,17,16,17
Emilia-Romagna,2,2,3,6,6,11,13,16,24,26,...,163,155,150,141,136,123,122,114,112,111


In [8]:
datasetRicoveratiTS = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-regioni_clean.csv")

#Recupero le colonne di interesse
TSricoverati_regioni = datasetRicoveratiTS[["data", "denominazione_regione", "ricoverati_con_sintomi"]].copy()

#Modifico l'indice, impostando la regione
TSricoverati_regioni.set_index("denominazione_regione", inplace=True)

#Effettuo il pivot della tabella ottenendo la time series
TSricoverati_regioni = TSricoverati_regioni.pivot_table(index='denominazione_regione', columns='data', values='ricoverati_con_sintomi')

TSricoverati_regioni.to_csv("Datasets/ProtezioneCivile/ts_regioni_ricoverati.csv")

TSricoverati_regioni.head()

data,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,...,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17
denominazione_regione,,,,,,,,,,,,,,,,,,,,,
Abruzzo,0,0,0,1,1,2,3,3,5,7,...,250,246,233,231,222,214,215,205,195,193
Basilicata,0,0,0,0,0,0,0,0,0,0,...,48,45,45,46,47,42,34,33,29,29
Calabria,0,0,0,0,0,0,0,0,0,0,...,73,68,66,64,65,64,59,56,56,56
Campania,0,0,0,2,2,3,4,4,11,11,...,418,415,435,436,414,402,387,341,351,342
Emilia-Romagna,10,15,20,36,56,86,127,148,187,256,...,1680,1604,1562,1537,1384,1165,861,763,740,721


In [9]:
datasetTerapiaTS = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-regioni_clean.csv")

#Recupero le colonne di interesse
TSterapia_daily_lombardia = datasetTerapiaTS[["data", "denominazione_regione", "terapia_intensiva"]].copy()

#Recupero la lombardia
TSterapia_daily_lombardia = TSterapia_daily_lombardia[TSterapia_daily_lombardia["denominazione_regione"]=='Lombardia']

#Calcola il valore giornaliero
TSterapia_daily_lombardia["daily_ti"] = TSterapia_daily_lombardia[['terapia_intensiva']].diff()

#Riempio il primo valore che è nullo
TSterapia_daily_lombardia.at[9,"daily_ti"]=TSterapia_daily_lombardia.at[9,'terapia_intensiva']

#elimino le colonne non utilizzate
TSterapia_daily_lombardia.drop(columns=["terapia_intensiva"],inplace=True)

#Modifico l'indice, impostando la regione
TSterapia_daily_lombardia.set_index("denominazione_regione", inplace=True)

#Effettuo il pivot della tabella ottenendo la time series
TSterapia_daily_lombardia = TSterapia_daily_lombardia.pivot_table(index='denominazione_regione', columns='data', values='daily_ti')

TSterapia_daily_lombardia.to_csv("ts_regioni_terapia_daily_lombardia.csv")


In [10]:
timeseriesPCNazionaleDataset = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-andamento-nazionale_clean.csv")

#Recupero le colonne di interesse
ricoveratiNazionali = timeseriesPCNazionaleDataset[["data", "ricoverati_con_sintomi"]].copy()

#Modifico l'indice, impostando la data
ricoveratiNazionali.set_index("data", inplace = True)

#Effettuo il pivot della tabella ottenendo la time series
ricoveratiNazionali = ricoveratiNazionali.T

ricoveratiNazionali.to_csv("Datasets/ProtezioneCivile/ts_nazioni_ricoverati.csv")

ricoveratiNazionali.head()

data,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,...,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17
ricoverati_con_sintomi,101,114,128,248,345,401,639,742,1034,1346,...,14636,13834,13618,13539,12865,12172,11453,10792,10400,10311


In [11]:
timeseriesPCNazionaleDataset = pd.read_csv("Datasets/ProtezioneCivile/dpc-covid19-ita-andamento-nazionale_clean.csv")

#Recupero le colonne di interesse
terapiaNazionali = timeseriesPCNazionaleDataset[["data", "terapia_intensiva"]].copy()

#Modifico l'indice, impostando la data
terapiaNazionali.set_index("data", inplace = True)

#Effettuo il pivot della tabella ottenendo la time series
terapiaNazionali = terapiaNazionali.T

terapiaNazionali.to_csv("Datasets/ProtezioneCivile/ts_nazioni_terapia.csv")

terapiaNazionali.head()

data,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,...,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17
terapia_intensiva,26,35,36,56,64,105,140,166,229,295,...,1168,1034,1027,999,952,893,855,808,775,762


### Dataset Limitazioni imposte dai governi

In [12]:
dataset_measures = pd.read_csv("Datasets/ContainmentMeasures/COVID 19 Containment measures data.csv")

#Prendiamo solo le colonne di nostro interesse e con pochi valori nulli
measures = dataset_measures[["Country", "Date Start", "Keywords"]].copy()

#Elimianiamo le tuple con valori nulli in quantoprive di significato
measures = measures.dropna(subset = ["Country", "Date Start", "Keywords"])

#Trasformiamo la data in datetime per poter avere uno standard su cui eseguire operazioni
measures["Date Start"] = pd.to_datetime(measures["Date Start"], format='%b %d, %Y')

measures.to_csv("Datasets/ContainmentMeasures/containment_measures_clean.csv", index=False)